<a href="https://colab.research.google.com/github/degartHub/nocountry-h12-25-equipo27-datascience/blob/main/H12_25_L_Equipo_27_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook para el proyecto de predicción de atrasos de vuelos - HACKATHON ONE

## Data Engineer (DE)

Sección para las tareas de Data Engineer.

Encargado: Ismael Cerda

### Selección y Limpieza de Datos

Base de datos obtenida de: https://www.kaggle.com/datasets/jimschacko/airlines-dataset-to-predict-a-delay?select=Airlines.csv

In [ ]:
import pandas as pd

url="https://raw.githubusercontent.com/degartHub/nocountry-h12-25-equipo27-datascience/refs/heads/main/data/Airlines.csv"
df = pd.read_csv(url)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           539383 non-null  int64 
 1   Airline      539383 non-null  object
 2   Flight       539383 non-null  int64 
 3   AirportFrom  539383 non-null  object
 4   AirportTo    539383 non-null  object
 5   DayOfWeek    539383 non-null  int64 
 6   Time         539383 non-null  int64 
 7   Length       539383 non-null  int64 
 8   Delay        539383 non-null  int64 
dtypes: int64(6), object(3)
memory usage: 37.0+ MB


In [ ]:
df["Time"].agg(["min", "max"])

,Time
min,10
max,1439


La base de datos cuenta con un total de 539.383 registros y un total de 9 columnas, siendo estas:

- <u>**id**</u>= Identifica la fila del registro.

- <u>**Airline**</u>= Aerolínea.

- <u>**Flight**</u>= Número de la aeronave.

- <u>**Airport From**</u>= Aeropuerto de salida.

- <u>**Airport To**</u>= Aeropuerto de destino.

- <u>**DayOfWeek**</u>= Día de la semana (en números).

- <u>**Time**</u>= Hora de salida medida en minutos a partir de la medianoche (rango de [10,1439], lo que podría ser el equivalente a un día).

- <u>**Lenght**</u>= Duración del vuelo en minutos.

- <u>**Delay**</u>= Con retraso (1), sin retraso (0).

In [ ]:
df.sample(n=5)

,id,Airline,Flight,AirportFrom,AirportTo,DayOfWeek,Time,Length,Delay
443836,443837,US,209,SMF,PHX,7,971,106,1
270272,270273,WN,3137,SMF,LAS,4,945,80,1
272572,272573,B6,1327,BOS,BWI,4,1065,93,1
305718,305719,WN,1745,MSY,HOU,6,925,70,0
86636,86637,WN,1021,MDW,BNA,1,410,85,1


Las colummnas a eliminar serán:
- ID: Es un identificador para la tabla en sí
- Flight: Identifica el número de avión, no es relevante.

In [ ]:
df = df.drop(columns=["id", "Flight"])

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539383 entries, 0 to 539382
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   Airline      539383 non-null  object
 1   AirportFrom  539383 non-null  object
 2   AirportTo    539383 non-null  object
 3   DayOfWeek    539383 non-null  int64 
 4   Time         539383 non-null  int64 
 5   Length       539383 non-null  int64 
 6   Delay        539383 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 28.8+ MB


## Feature Architect (FA)

Sección para las tareas de Feature Architect

Encargado: Eduardo Ayala

### Ingeniería de Atributos

**Acerca de los datos faltantes**

Nos faltan fechas y distancia recorrida en km

*   Si observamos los valores mínimos y máximos de la columna `Time` se ve que están en el rango (10, 1439), que corresponde aproximadamente a los minutos que tiene un día completo, y, aparte, hay una columna `Lenght` con el tiempo de vuelo. Como son ~540 mil vuelos no se puede asumir que son todos del mismo día, es algo más parecido a los vuelos de 1 mes en USA. Así que se crearán las fechas de partida de manera sintética para 1 mes.
*   Adicionalmente a los anterior, nos piden la distancia en kilómetros, pero tenemos el tiempo de vuelo; así que haremos la conversión estimando $800\frac{km}{\text{hr}}$ (velocidad promedio de un vuelo comercial).



Sumado a lo anterior, vamos a renombrar las columnas de acuerdo al contrato con backend.

In [ ]:
import numpy as np
import pandas as pd

# ---------------------------------------------------------------------
#  CONFIGURACIÓN
# ---------------------------------------------------------------------
np.random.seed(42)

VELOCIDAD_PROMEDIO_KMH = 800

start_date = pd.to_datetime('2018-12-01')
end_date = pd.to_datetime('2018-12-31')

# ---------------------------------------------------------------------
#  FECHAS ALEATORIAS (DÍA)
# ---------------------------------------------------------------------
random_days = np.random.randint(
    0,
    (end_date - start_date).days + 1,
    size=len(df)
)

df['FlightDate'] = (
    start_date + pd.to_timedelta(random_days, unit='D')
).normalize()

# ---------------------------------------------------------------------
#  FECHA Y HORA DE SALIDA
#  Time = minutos desde medianoche
# ---------------------------------------------------------------------
df['DepartureDateTime'] = (
    df['FlightDate'] + pd.to_timedelta(df['Time'], unit='m')
)

# ---------------------------------------------------------------------
#  RECONSTRUIR DURACIÓN DESDE DISTANCIA
#  Distance_km = distancia en km
# ---------------------------------------------------------------------
df['duration_min'] = (
    df['Distance_km'] / VELOCIDAD_PROMEDIO_KMH
) * 60

# ---------------------------------------------------------------------
#  FECHA Y HORA DE LLEGADA
# ---------------------------------------------------------------------
df['ArrivalDateTime'] = (
    df['DepartureDateTime'] + pd.to_timedelta(df['duration_min'], unit='m')
)

# ---------------------------------------------------------------------
#  RENOMBRE DE COLUMNAS SEGÚN CONTRATO BACKEND
# ---------------------------------------------------------------------
df = df.rename(columns={
    'Airline': 'aerolinea',
    'AirportFrom': 'origen',
    'AirportTo': 'destino',
    'Distance_km': 'distancia_km'
})

# ---------------------------------------------------------------------
#  FECHA PARTIDA FORMATO ISO-8601
# ---------------------------------------------------------------------
df['fecha_partida'] = df['DepartureDateTime'].dt.strftime('%Y-%m-%dT%H:%M:%S')

# ---------------------------------------------------------------------
#  DATAFRAME FINAL PARA BACKEND
# ---------------------------------------------------------------------
df_backend = df[
    ['aerolinea', 'origen', 'destino', 'fecha_partida', 'distancia_km']
]

# ---------------------------------------------------------------------
#  VERIFICACIÓN
# ---------------------------------------------------------------------
print(df_backend.head())
print(df_backend.dtypes)

In [ ]:
# ---------------------------------------------------------------------
#  PARTE 2: CREAR VARIABLES TEMPORALES
# ---------------------------------------------------------------------
# A partir de columnas datetime ya existentes:
# - FlightDate
# - DepartureDateTime
# - ArrivalDateTime

# ---------------------------------------------------------------------
#  VARIABLES DE SALIDA
# ---------------------------------------------------------------------
df['DepartureHour'] = df['DepartureDateTime'].dt.hour
df['DepartureDayOfWeek'] = df['DepartureDateTime'].dt.dayofweek

# ---------------------------------------------------------------------
#  VARIABLES DE LLEGADA
# ---------------------------------------------------------------------
df['ArrivalHour'] = df['ArrivalDateTime'].dt.hour
df['ArrivalDayOfWeek'] = df['ArrivalDateTime'].dt.dayofweek

# ---------------------------------------------------------------------
#  VERIFICACIÓN RÁPIDA
# ---------------------------------------------------------------------
print(
    df[
        [
            'DepartureDateTime',
            'DepartureHour',
            'DepartureDayOfWeek',
            'ArrivalDateTime',
            'ArrivalHour',
            'ArrivalDayOfWeek'
        ]
    ].head()
)

    DepartureDateTime  DepartureHour  DepartureDayOfWeek     ArrivalDateTime  \
0 2018-12-07 00:15:00              0                   4 2018-12-07 03:40:00   
1 2018-12-20 00:15:00              0                   3 2018-12-20 03:57:00   
2 2018-12-29 00:20:00              0                   5 2018-12-29 03:05:00   
3 2018-12-15 00:20:00              0                   5 2018-12-15 03:35:00   
4 2018-12-11 00:30:00              0                   1 2018-12-11 03:52:00   

   ArrivalHour  ArrivalDayOfWeek  
0            3                 4  
1            3                 3  
2            3                 5  
3            3                 5  
4            3                 1  


In [ ]:
from sklearn.preprocessing import OneHotEncoder

# ---------------------------------------------------------------------
#  PARTE 3: ONE-HOT ENCODING DE VARIABLES CATEGÓRICAS
# ---------------------------------------------------------------------

# Variables categóricas a codificar
categorical_features = [
    'Airline',
    'AirportFrom',
    'AirportTo'
]

# Inicializar encoder
# handle_unknown='ignore' permite usar el modelo en producción
ohe = OneHotEncoder(
    sparse_output=False,
    handle_unknown='ignore'
)

# Aplicar One-Hot-Encoding
X_cat = ohe.fit_transform(df[categorical_features])

# Convertir a DataFrame para mejor legibilidad
X_cat = pd.DataFrame(
    X_cat,
    columns=ohe.get_feature_names_out(categorical_features),
    index=df.index
)

# ---------------------------------------------------------------------
#  VERIFICACIÓN RÁPIDA
# ---------------------------------------------------------------------
print("Shape variables categóricas codificadas:", X_cat.shape)
print(X_cat.head())

Shape variables categóricas codificadas: (539383, 604)
   Airline_9E  Airline_AA  Airline_AS  Airline_B6  Airline_CO  Airline_DL  \
0         0.0         0.0         0.0         0.0         1.0         0.0   
1         0.0         0.0         0.0         0.0         0.0         0.0   
2         0.0         1.0         0.0         0.0         0.0         0.0   
3         0.0         1.0         0.0         0.0         0.0         0.0   
4         0.0         0.0         1.0         0.0         0.0         0.0   

   Airline_EV  Airline_F9  Airline_FL  Airline_HA  ...  AirportTo_TXK  \
0         0.0         0.0         0.0         0.0  ...            0.0   
1         0.0         0.0         0.0         0.0  ...            0.0   
2         0.0         0.0         0.0         0.0  ...            0.0   
3         0.0         0.0         0.0         0.0  ...            0.0   
4         0.0         0.0         0.0         0.0  ...            0.0   

   AirportTo_TYR  AirportTo_TYS  AirportTo_

**Uso de escalado según el modelo**

El escalado de variables numéricas no siempre es necesario y su aplicación depende del tipo de modelo de machine learning que se esté utilizando. En el caso de la **Regresión Logística**, el escalado **sí es necesario**, ya que este modelo es sensible a la magnitud de las variables numéricas. Al basarse en combinaciones lineales y procesos de optimización por gradiente, el escalado permite un entrenamiento más estable y una correcta asignación de pesos a cada variable. Por otro lado, **Random Forest** **no requiere escalado**, ya que al estar basado en árboles de decisión realiza divisiones por umbrales y no utiliza distancias ni gradientes, por lo que la escala de las variables no influye en su desempeño.

**Aplicación en el código**

En el código se crearon dos conjuntos de datos distintos para respetar los
requisitos de cada modelo. Para Regresión Logística, las variables numéricas
(`Length`, `DepartureHour`, `ArrivalHour`) fueron escaladas utilizando
`StandardScaler`. Para Random Forest, las mismas variables se utilizaron en su
escala original. Ambos conjuntos de datos comparten las variables categóricas
transformadas mediante One-Hot Encoding, y los objetos de transformación fueron
guardados para su uso posterior en producción.

In [ ]:
from sklearn.preprocessing import StandardScaler

# ---------------------------------------------------------------------
#  PARTE 4: ESCALADO Y DATASETS POR MODELO
# ---------------------------------------------------------------------

# Variable objetivo
y = df['Delay']

# Variables numéricas seleccionadas
numeric_features = [
    'Length',
    'DepartureHour',
    'ArrivalHour'
]

X_num = df[numeric_features]

# ---------------------------------------------------------------------
#  DATASET PARA RANDOM FOREST (SIN ESCALADO)
# ---------------------------------------------------------------------
X_rf = pd.concat([X_num, X_cat], axis=1)

print("Shape X_rf:", X_rf.shape)

# ---------------------------------------------------------------------
#  DATASET PARA REGRESIÓN LOGÍSTICA (CON ESCALADO)
# ---------------------------------------------------------------------
scaler = StandardScaler()

X_num_scaled = scaler.fit_transform(X_num)

X_num_scaled = pd.DataFrame(
    X_num_scaled,
    columns=numeric_features,
    index=df.index
)

X_logreg = pd.concat([X_num_scaled, X_cat], axis=1)

print("Shape X_logreg:", X_logreg.shape)

Shape X_rf: (539383, 607)
Shape X_logreg: (539383, 607)


In [ ]:
import joblib

# ---------------------------------------------------------------------
#  PARTE 5: GUARDAR OBJETOS DE TRANSFORMACIÓN
# ---------------------------------------------------------------------
# Estos objetos se reutilizan en inferencia para garantizar
# consistencia entre entrenamiento y producción

joblib.dump(scaler, 'scaler_logreg.pkl')
joblib.dump(ohe, 'onehot_encoder.pkl')

print("Objetos de transformación guardados correctamente")

Objetos de transformación guardados correctamente


**Reducción de memoria en el dataset (opcional)**

Para poder entrenar los modelos con todo el dataset sin saturar la RAM, realizamos los siguientes cambios:

1. Cambio de tipos de datos:
   - Variables numéricas (Length, DepartureHour, ArrivalHour) convertidas de float64 a float32
     - Esto reduce a la mitad el tamaño de cada celda.
   - Variables categóricas One-Hot (0.0 / 1.0) convertidas a uint8
     - Cada celda ocupa solo 1 byte en lugar de 8 bytes.

2. Uso de sparse matrices:
   - One-Hot Encoding genera muchas columnas con ceros.
   - Convertir el DataFrame a sparse.csr_matrix evita almacenar los ceros.
   - Esto ahorra memoria y permite entrenar Regresión Logística sin sample.

3. Escalado eficiente:
   - Para Regresión Logística, las columnas numéricas se escalaron con StandardScaler(with_mean=False)
   - with_mean=False es obligatorio para sparse matrices, evitando densificar el dataset.

Con estos cambios, el dataset pasó de varios GB a menos de 1 GB, permitiendo entrenar con todo el dataset y manteniendo un rendimiento estable.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy import sparse

# ---------------------------------------------------------------------
#  REDUCCIÓN DE MEMORIA
# ---------------------------------------------------------------------

# Variables numéricas
numeric_features = ['Length', 'DepartureHour', 'ArrivalHour']

# Convertir numéricas a float32
X_logreg[numeric_features] = X_logreg[numeric_features].astype('float32')
X_rf[numeric_features] = X_rf[numeric_features].astype('float32')

# Convertir variables categóricas One-Hot a uint8
X_logreg[X_cat.columns] = X_cat.astype('uint8')
X_rf[X_cat.columns] = X_cat.astype('uint8')

print("Tipos de datos convertidos. Memoria reducida.")

# ---------------------------------------------------------------------
#  OPCIONAL: convertir a sparse matrices para ahorrar más memoria
# ---------------------------------------------------------------------
# Para Regresión Logística
X_logreg_sparse = sparse.csr_matrix(X_logreg.values)

# Para Random Forest (opcional, RF puede manejar DataFrame normal)
X_rf_sparse = sparse.csr_matrix(X_rf.values)

print("Datasets convertidos a sparse. Listos para entrenar.")
print("X_logreg_sparse shape:", X_logreg_sparse.shape)
print("X_rf_sparse shape:", X_rf_sparse.shape)

# ---------------------------------------------------------------------
#  ESCALADO DE VARIABLES NUMÉRICAS PARA LOGREG
# ---------------------------------------------------------------------
scaler = StandardScaler(with_mean=False)  # with_mean=False es obligatorio para sparse

X_logreg_scaled = X_logreg_sparse.copy()
X_logreg_scaled[:, :len(numeric_features)] = scaler.fit_transform(X_logreg_sparse[:, :len(numeric_features)])

print("Variables numéricas escaladas. Dataset listo para Regresión Logística.")

Tipos de datos convertidos. Memoria reducida.
Datasets convertidos a sparse. Listos para entrenar.
X_logreg_sparse shape: (539383, 607)
X_rf_sparse shape: (539383, 607)
Variables numéricas escaladas. Dataset listo para Regresión Logística.


## Machine Learning Engineer (MLE)

Sección para las tareas de Machine Learning Engineer

Encargado: Luis Jácome

### Entrenamiento y Evaluación base

##Split de Datos (Train/Test) con un random_state fijo
En esta primera tarea separaremos los datos en entrenamiento y prueba con el objetivo de que sea reproducible, se encuentre balanceada y lista para entrenar al modelo al cual se le asignará el nombre de champion.

In [ ]:
from sklearn.model_selection import train_test_split

# --------------------------------------------------
# SPLIT TRAIN / TEST (MLE - Tarea 1)
# --------------------------------------------------

X_train, X_test, y_train, y_test = train_test_split(
    X_logreg_scaled,
    y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

# Verificación rápida
print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)

print("\nDistribución Delay (train):")
print(y_train.value_counts(normalize=True))

print("\nDistribución Delay (test):")
print(y_test.value_counts(normalize=True))


Train shape: (431506, 607)
Test shape: (107877, 607)

Distribución Delay (train):
Delay
0    0.554558
1    0.445442
Name: proportion, dtype: float64

Distribución Delay (test):
Delay
0    0.554558
1    0.445442
Name: proportion, dtype: float64


## Entrenar el modelo seleccionado con parametros por defecto
Entrenaremos un modelo baseline usando los datos ya escalados y dejar el modelo listo para inferencia.
Se utilizará Logistic Regression para el entrenamiento ya que es un modelo interpretable, rápido, robusto y adecuado como baselinepara clasificación binaria.

In [ ]:
from sklearn.linear_model import LogisticRegression
import joblib

# --------------------------------------------------
# ENTRENAMIENTO MODELO BASE - LOGISTIC REGRESSION
# --------------------------------------------------

champion = LogisticRegression(
    random_state=42,
    max_iter=1000,
    solver='liblinear'
)

champion.fit(X_train, y_train)

print("Modelo Logistic Regression entrenado correctamente")

# --------------------------------------------------
# GUARDAR MODELO PARA PRODUCCIÓN
# --------------------------------------------------

joblib.dump(champion, 'champion.pkl')

print("Modelo guardado como champion.pkl")


Modelo Logistic Regression entrenado correctamente
Modelo guardado como champion.pkl


## Evaluación del modelo
evaluaremos el modelo baseline mediante métricas estándar de clasificación binaria (Accuracy, Precision, Recall y F1-score) tanto en el conjunto de entrenamiento como de prueba, utilizando un umbral de decisión por defecto de 0.5

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# --------------------------------------------------
# PREDICCIONES
# --------------------------------------------------
y_train_pred = champion.predict(X_train)
y_test_pred = champion.predict(X_test)
# --------------------------------------------------
# MÉTRICAS TRAIN
# --------------------------------------------------
train_metrics = {
    "Accuracy": accuracy_score(y_train, y_train_pred),
    "Precision": precision_score(y_train, y_train_pred),
    "Recall": recall_score(y_train, y_train_pred),
    "F1-Score": f1_score(y_train, y_train_pred)
}
# --------------------------------------------------
# MÉTRICAS TEST
# --------------------------------------------------
test_metrics = {
    "Accuracy": accuracy_score(y_test, y_test_pred),
    "Precision": precision_score(y_test, y_test_pred),
    "Recall": recall_score(y_test, y_test_pred),
    "F1-Score": f1_score(y_test, y_test_pred)
}
# --------------------------------------------------
# MOSTRAR RESULTADOS
# --------------------------------------------------
print("Métricas Train:")
for k, v in train_metrics.items():
    print(f"{k}: {v:.4f}")

print("\nMétricas Test:")
for k, v in test_metrics.items():
    print(f"{k}: {v:.4f}")

Métricas Train:
Accuracy: 0.6453
Precision: 0.6318
Recall: 0.4881
F1-Score: 0.5507

Métricas Test:
Accuracy: 0.6458
Precision: 0.6326
Recall: 0.4887
F1-Score: 0.5514


## Validar que el modelo no tenga overfitting excesivo
La ausencia de overfitting se evidencia en la similitud casi exacta entre las métricas de entrenamiento y prueba. Las diferencias inferiores al 0.1% indican que el modelo generaliza correctamente y no presenta alta varianza.

## Seleccionar el umbral de probabilidad óptimo para la clasificación


In [ ]:
# Probabilidad de clase positiva (Delay = 1)
y_proba_test = champion.predict_proba(X_test)[:, 1]

In [ ]:
# Rango de umbrales a evaluar
thresholds = np.arange(0.1, 0.9, 0.05)

results = []

for threshold in thresholds:
    y_pred_threshold = (y_proba_test >= threshold).astype(int)

    precision = precision_score(y_test, y_pred_threshold)
    recall = recall_score(y_test, y_pred_threshold)
    f1 = f1_score(y_test, y_pred_threshold)

    results.append({
        "Threshold": threshold,
        "Precision": precision,
        "Recall": recall,
        "F1_score": f1
    })

# Resultados en DataFrame
threshold_df = pd.DataFrame(results)

threshold_df


,Threshold,Precision,Recall,F1_score
0,0.10,0.445741,0.999854,0.616599
1,0.15,0.449290,0.996858,0.619409
2,0.20,0.459645,0.981416,0.626070
3,0.25,0.477493,0.945227,0.634473
4,0.30,0.502313,0.883649,0.640520
5,0.35,0.530676,0.798140,0.637490
6,0.40,0.562722,0.697896,0.623062
7,0.45,0.596207,0.591493,0.593841
8,0.50,0.632556,0.488690,0.551393
9,0.55,0.673274,0.396541,0.499116


In [ ]:
# selección del umbral óptimo (máximo F1)
best_threshold_row = threshold_df.loc[threshold_df['F1_score'].idxmax()]
best_threshold_row

,4
Threshold,0.300000
Precision,0.502313
Recall,0.883649
F1_score,0.640520


In [ ]:
# guardado del umbral óptimo
best_threshold = best_threshold_row['Threshold']

print(f"Umbral óptimo seleccionado: {best_threshold:.2f}")


Umbral óptimo seleccionado: 0.30


In [ ]:
# Comparación directa vs umbral 0.5
# Predicción con umbral por defecto
y_pred_default = (y_proba_test >= 0.5).astype(int)

# Predicción con umbral óptimo
y_pred_optimal = (y_proba_test >= best_threshold).astype(int)

comparison = pd.DataFrame({
    "Metric": ["Precision", "Recall", "F1_score"],
    "Threshold_0.5": [
        precision_score(y_test, y_pred_default),
        recall_score(y_test, y_pred_default),
        f1_score(y_test, y_pred_default)
    ],
    "Optimal_Threshold": [
        precision_score(y_test, y_pred_optimal),
        recall_score(y_test, y_pred_optimal),
        f1_score(y_test, y_pred_optimal)
    ]
})

comparison


,Metric,Threshold_0.5,Optimal_Threshold
0,Precision,0.632556,0.502313
1,Recall,0.488690,0.883649
2,F1_score,0.551393,0.640520


Al evaluar distintos umbrales de decisión, se identificó un umbral óptimo que maximiza el F1-score el cual es 0.3. Este ajuste incrementa significativamente la capacidad del modelo para detectar vuelos retrasados (Recall ≈ 88%), a costa de una reducción moderada en Precision, logrando un mejor equilibrio general sin necesidad de reentrenar el modelo.

## Machine Learning Operations (MLOps)

Sección para las tareas de Machine Learning Operations

Encargado: Nicolás Staffelbach

### Microservicio Python

#### Validación de las versiones de las librerias en Colab


```
!pip show fastapi...
```
Este código tiene como fin el saber las versiones de las librerias utilizadas en el entorno Google Colab, para la creación del archivo `requirements.txt` para garantizar el funcionamiento del modelo en producción.


In [ ]:
!pip show fastapi scikit-learn pandas numpy joblib uvicorn pydantic

Name: fastapi
Version: 0.123.10
Summary: FastAPI framework, high performance, easy to learn, fast to code, ready for production
Home-page: https://github.com/fastapi/fastapi
Author: 
Author-email: =?utf-8?q?Sebasti=C3=A1n_Ram=C3=ADrez?= <tiangolo@gmail.com>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: annotated-doc, pydantic, starlette, typing-extensions
Required-by: google-adk, gradio
---
Name: scikit-learn
Version: 1.6.1
Summary: A set of python modules for machine learning and data mining
Home-page: https://scikit-learn.org
Author: 
Author-email: 
License: BSD 3-Clause License

 Copyright (c) 2007-2024 The scikit-learn developers.
 All rights reserved.

 Redistribution and use in source and binary forms, with or without
 modification, are permitted provided that the following conditions are met:

 * Redistributions of source code must retain the above copyright notice, this
   list of conditions and the following disclaimer.

 * Redistributions in binary for

#### Script de carga del modelo

En esta sección se desarrolla la creación del pipeline de carga del encoder, scaler y modelo, para su posterior uso en producción, garantizando el uso de los mismos objetos utilizados para el entrenamiento del modelo. Y también garantizando la optimización de la API.

In [ ]:
"""
import joblib
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

encoder = joblib.load ('onehot_encoder.pkl')
scaler = joblib.load('scaler_logreg.pkl')
trained_model = joblib.load('champion.pkl')

#==========================
# 1. Definición de columnas
#==========================

 categorical_features = [
    "aerolinea",
    "origen",
    "destino",
    "dia_semana"
]

numerical_features = [
  "hora_salida",
  "distancia_km"
]


#====================
# 2. Preprocesamiento
#====================

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", encoder, categorical_features),
        ("numerical", scaler, numerical_features)
    ],
    remainder="drop"
)

#============
# 2. Pipeline
#============

pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", trained_model)
    ]
)

joblib.dump(pipeline, "flight-delay.joblib")
"""


'\nimport joblib\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.compose import ColumnTransformer\n\nencoder = joblib.load (\'onehot_encoder.pkl\')\nscaler = joblib.load(\'scaler_logreg.pkl\')\ntrained_model = joblib.load(\'champion.pkl\')\n\n#==========================\n# 1. Definición de columnas\n#==========================\n\n categorical_features = [\n    "aerolinea",\n    "origen",\n    "destino",\n    "dia_semana"\n]\n\nnumerical_features = [\n  "hora_salida",\n  "distancia_km"\n]\n\n\n#====================\n# 2. Preprocesamiento\n#====================\n\npreprocessor = ColumnTransformer(\n    transformers=[\n        ("categorical", encoder, categorical_features),\n        ("numerical", scaler, numerical_features)\n    ],\n    remainder="drop"\n)\n\n#============\n# 2. Pipeline\n#============\n\npipeline = Pipeline(\n    steps=[\n        ("preprocessor", preprocessor),\n        ("classifier", trained_model)\n    ]\n)\n\njoblib.dump(pipeline, "flight-delay.joblib")\n'

## Data Analyst (DA)

Sección para las tareas de Data Analyst

Encargado: David Aragón

### Análisis de Datos Exploratorio EDA